In [ ]:
# Test run of the CpG Net API.
# Author: Jack Duryea

In [16]:
from CpG_Net import CpGNet
import numpy as np

# Automatic reloading so we don't have to restart the kernel each time we make a change
# to CpGNet when we are debuggin
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Step 1: Collect some data

In [17]:

# Some bins with 2 cpgs
bin1 = np.array([[1,0],[0,-1],[-1,1],[0,0]],dtype=float)
bin2 = np.array([[1,0],[1,0],[-1,1],[0,0],[0,1],[1,1],[0,0]],dtype=float)
bin3 = np.array([[-1,1],[0,-1],[-1,1],[0,0]],dtype=float)
bin4 = np.array([[1,-1],[0,1],[-1,1],[1,0]],dtype=float)
bin5 = np.array([[1,1],[0,0],[0,1],[1,1]],dtype=float)
bin6 = np.array([[1,1],[1,1],[0,1],[1,0]],dtype=float)
bin7 = np.array([[0,0],[1,0],[0,1],[1,1]],dtype=float)

# Collection of bins
bins = [bin1,bin2,bin3,bin4, bin5, bin6, bin7]

### Step 2: Create a model and train on the data. CpGNet will take care of  feature extraction and all that other boring stuff, it just works!

In [18]:
net = CpGNet(cpgDensity=2)
net.train(bins)

100%|██████████| 3/3 [00:00<00:00, 326.91it/s]


encoded vector: [ 0.  0.  1.  0.  0.  0.  1.  1.  1.]
encoded vector normalized [[ 0.    0.    0.25  0.    0.    0.    0.25  0.25  0.25]]
encoded vector: [ 0.  1.  1.  0.  0.  0.  1.  0.  1.]
encoded vector normalized [[ 0.    0.25  0.25  0.    0.    0.    0.25  0.    0.25]]
encoded vector: [ 0.  0.  1.  0.  1.  0.  1.  0.  1.]
encoded vector normalized [[ 0.    0.    0.25  0.    0.25  0.    0.25  0.    0.25]]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 1000)              16000     
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 1000)              0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 800)               800

### Step 3: Impute!

In [11]:
net.impute(bin1)

array([[ 1.        ,  0.        ],
       [ 0.        ,  0.48987553],
       [ 0.49581507,  1.        ],
       [ 0.        ,  0.        ]])

In [12]:
binA = np.array([[0,0],[0,0],[0,-1],[0,0],[0,0]],dtype=float)

In [13]:
binB = np.array([[1,1],[1,1],[1,-1],[1,1],[1,1]],dtype=float)

In [14]:
net.impute(binB)

array([[ 1.        ,  1.        ],
       [ 1.        ,  1.        ],
       [ 1.        ,  0.49002632],
       [ 1.        ,  1.        ],
       [ 1.        ,  1.        ]])

### Step 4: Publish?